# Read & update a standard JSON template

In this notebook we cover two ways to read and update an existing COOLEST template file:

- _Option 1_: using standard JSON utilities;

- _Option 2_: using the dedicated COOLEST Python interface.

__author__: @aymgal

__last update__: 01/06/23

In [1]:
import os
from pprint import pprint

In [2]:
TEMPLATE_NAME = 'coolest_template'
TEMPLATE_DIR = 'template_dir'

## _Option 1:_ Load the JSON as a nested dictionnary

This option only requires to the standard python module `json`. This can be useful for example when querying the template online for database purposes.

It allows to update 'by hand' the content of the JSON file, following the syntax of python containers (essentially nested `dict`s and `list`s).

In [3]:
import json

### Load the JSON as standard dictionnary

In [4]:
with open(os.path.join(os.getcwd(), TEMPLATE_DIR, TEMPLATE_NAME + '.json'), 'r') as f:
    content = json.load(f)

### Explore its content

In [5]:
# print the lensing entities
pprint(content['lensing_entities'])

[{'mass_model': [{'parameters': {'gamma_ext': {'fixed': False,
                                               'point_estimate': {'value': 0.07},
                                               'posterior_stats': {'mean': None,
                                                                   'median': None,
                                                                   'percentile_16th': None,
                                                                   'percentile_84th': None},
                                               'prior': {'type': None}},
                                 'phi_ext': {'fixed': False,
                                             'point_estimate': {'value': None},
                                             'posterior_stats': {'mean': None,
                                                                 'median': None,
                                                                 'percentile_16th': None,
                                          

In [6]:
# example for accessing some of the content
pprint(content['lensing_entities'][1]['mass_model'][0]['parameters']['theta_E'])

{'fixed': False,
 'point_estimate': {'value': None},
 'posterior_stats': {'mean': None,
                     'median': None,
                     'percentile_16th': None,
                     'percentile_84th': None},
 'prior': {'type': None}}


### Add a prior on a given parameter

In [7]:
# type of prior
content['lensing_entities'][1]\
    ['mass_model'][0]['parameters']['theta_E']\
    ['prior']['type'] = 'gaussian'

# mean
content['lensing_entities'][1]\
    ['mass_model'][0]['parameters']['theta_E']\
    ['prior']['mean'] = 1.0

# width
content['lensing_entities'][1]\
    ['mass_model'][0]['parameters']['theta_E']\
    ['prior']['width'] = 0.01

### Assign a point estimate to a given parameter

In [8]:
# here we add a gaussian prior on the source effective radius
content['lensing_entities'][1]\
    ['light_model'][0]['parameters']['theta_eff']\
    ['value'] = 0.8

### Assign statistics about the posterior distribution of a parameter

In [9]:
# here we add posterior statistics on the external shear strength
content['lensing_entities'][0]\
    ['mass_model'][0]['parameters']['gamma_ext']\
    ['posterior_stats']['mean'] = 0.03

content['lensing_entities'][0]\
    ['mass_model'][0]['parameters']['gamma_ext']\
    ['posterior_stats']['median'] = 0.031

content['lensing_entities'][0]\
    ['mass_model'][0]['parameters']['gamma_ext']\
    ['posterior_stats']['percentile_16th'] = 0.02

content['lensing_entities'][0]\
    ['mass_model'][0]['parameters']['gamma_ext']\
    ['posterior_stats']['percentile_84th'] = 0.03

### Save the current content as an updated JSON file

In [10]:
# uncomment for saving the updated JSON file
with open(os.path.join(os.getcwd(), TEMPLATE_DIR, 'updated_via_json.json'), 'w') as f:
    f.write(json.dumps(content, sort_keys=True, indent=2))
    
# Note: you can also update the original JSON template if you wish

## _Option 2:_ Use the `COOLEST` Python interface

The `coolest.template` interface allows to manipulate the content of the JSON file as a hierachy of python objects, and to updated fields with new instances of objects.

In [11]:
from coolest.template.json import JSONSerializer
from coolest.template.classes.probabilities import GaussianPrior
from coolest.template.classes.parameter import PointEstimate
from coolest.template.classes.probabilities import PosteriorStatistics

### Decode the JSON file

In [12]:
decoder = JSONSerializer(os.path.join(os.getcwd(), TEMPLATE_DIR, TEMPLATE_NAME), indent=2)
coolest_object = decoder.load()

### Explore the content, print some attributes

In [13]:
print("Content:")
pprint(vars(coolest_object))

Content:
{'coordinates_origin': <coolest.template.classes.coordinates.CoordinatesOrigin object at 0x121c21d90>,
 'cosmology': <coolest.template.classes.cosmology.Cosmology object at 0x121c21760>,
 'exclude_keys': ['documentation',
                  'id',
                  'latex_str',
                  'units',
                  'definition_range',
                  'exclude_keys'],
 'instrument': <coolest.template.classes.instrument.Instrument object at 0x121c21b20>,
 'lensing_entities': [<coolest.template.classes.mass_field.MassField object at 0x121c21a30>,
 'meta': {},
 'mode': 'MAP',
 'observation': <coolest.template.classes.observation.Observation object at 0x121c21d60>,
 'standard': 'COOLEST'}


In [14]:
lensing_entities = coolest_object.lensing_entities
print("Attributes of the lens model:")
pprint(lensing_entities)

Attributes of the lens model:


In [15]:
lens_galaxy = lensing_entities[1]
print("Attributes of the first galaxy in the lens model:")
pprint(vars(lens_galaxy))

Attributes of the first galaxy in the lens model:
{'documentation': 'Class that defines a Galaxy entity (lens galaxy and/or '
                  'source galaxy).\n'
                  '\n'
                  '    Parameters\n'
                  '    ----------\n'
                  '    name : str\n'
                  '        Name associated to the galaxy.\n'
                  '    redshift : float\n'
                  '        Redshift of the galaxy, if any.\n'
                  '    light_model : LightModel, optional\n'
                  '        Light model instance describing the surface '
                  'brightness of the galaxy, by default None\n'
                  '    mass_model : MassModel, optional\n'
                  '        Mass model instance describing the mass '
                  'distribution of the galaxy, by default None',
 'light_model': [<coolest.template.classes.profiles.light.Sersic object at 0x121c16f70>,
 'mass_model': [<coolest.template.classes.profiles.mass.

In [16]:
lens_profile = lens_galaxy.mass_model[0]
print("Attributes of its first mass profile:")
pprint(vars(lens_profile))

Attributes of its first mass profile:
{'documentation': 'Power-law elliptical mass density profile, \n'
                  '    also called the Elliptical Power-law profile (EPL).\n'
                  '\n'
                  '    This profile is described by the following parameters:\n'
                  '\n'
                  "    - 'gamma': logarithmic radial slope\n"
                  "    - 'theta_E': Einstein radius\n"
                  "    - 'q': axis ratio (semi-major axis / semi-minor axis)\n"
                  "    - 'phi': position angle\n"
                  "    - 'center_x': position along the x coordinate\n"
                  "    - 'center_y': position along the y coordinate",
 'id': 'galaxy_1-mass_model-PEMD_0',
 'parameters': {'center_x': <coolest.template.classes.parameter.NonLinearParameter object at 0x121c1c610>,
                'center_y': <coolest.template.classes.parameter.NonLinearParameter object at 0x121c16640>,
                'gamma': <coolest.template.classes

### Add a prior on a given parameter

In [17]:
# here we add a gaussian prior on the Einstein radius
prior = GaussianPrior(mean=1.0, width=0.01)

coolest_object.lensing_entities[1].mass_model[0].parameters['theta_E'].set_prior(prior)

### Assign a point estimate to a given parameter

In [18]:
# here we add a gaussian prior on the source effective radius
estimate = PointEstimate(value=0.8)

coolest_object.lensing_entities[2].light_model[0].parameters['theta_eff'].set_point_estimate(estimate)

### Assign statistics about the posterior distribution of a parameter

In [19]:
# here we add posterior statistics on the shear strength
posterior \
    = PosteriorStatistics(mean=0.03, median=0.031, 
                          percentile_16th=0.02, percentile_84th=0.03)

coolest_object.lensing_entities[0].mass_model[0].parameters['gamma_ext'].set_posterior(posterior)

### Encode the updated classes as a new template

In [20]:
encoder = JSONSerializer(os.path.join(os.getcwd(), TEMPLATE_DIR, 'updated_via_coolest'), 
                         obj=coolest_object, indent=2)

# uncomment for dumping the content to the JSON file
coolest_object_encoded = encoder.dump_simple()

# Note: you can also update the original JSON template if you wish